In [1]:
!pip install -q transformers datasets
!pip install pyvi
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 70.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.1 MB/s eta 0:00:00


In [3]:
%cd /kaggle/input/intent-customer-new/intent_dialogue/call_intent_analysis/intent_customer

/kaggle/input/intent-customer-new/intent_dialogue/call_intent_analysis/intent_customer


In [4]:
import pandas as pd
import json

def convert_excel_json(path):
    # Giả sử 'df' là DataFrame của bạn đã được tải lên từ file Excel
    df = pd.read_excel(path)

    # Thay thế giá trị NaN trong cột "intent" bằng "khác"
    df['intent'] = df['intent'].fillna("khác")
#     df = df.loc[df['intent'] != 'khác']
    # Chuyển DataFrame thành danh sách các đối tượng JSON
    # Lọc ra những hàng mà giá trị trong cột 'intent' không phải là 'khác'
#     df = df.loc[df['intent'] != 'phản ánh các vấn đề khác']
    df['intent'] = df['intent'].replace('phản ánh các vấn đề khác', 'khác')
    df['intent'] = df['intent'].replace('Khách hàng thắc mắc khiếu nại về việc gia hạn gói cước', 'Thông tin gia hạn gói cước đang sử dụng')
    list_json = df[['text', 'intent']].to_dict(orient='records')

#     print(list_json)
    name_file= path.split(".")[0].split("/")[-1]+".json"
    with open("/kaggle/working/"+name_file, 'w', encoding='utf-8') as f:
        json.dump(list_json, f, ensure_ascii=False)
    


path1= "/kaggle/input/topic-50-new-v2/train_data.xlsx"
path2= "/kaggle/input/topic-50-new-v2/test_data.xlsx"
path3= "/kaggle/input/topic-50-new-v2/test_2.xlsx"
convert_excel_json(path1)
convert_excel_json(path2)
convert_excel_json(path3)

In [17]:
# !python train.py

In [18]:
# from imblearn.over_sampling import RandomOverSampler
# import pandas as pd

# def resample_df(df):
#     # Chuẩn bị dữ liệu
#     X = df['text']
#     y = df['intent']

#     # Tạo đối tượng RandomOverSampler
#     ros = RandomOverSampler(sampling_strategy={label: 300 for label in y.unique()}, random_state=42)

#     # Thực hiện upsampling
#     X_resampled, y_resampled = ros.fit_resample(X.to_frame(), y)

#     # Tạo lại DataFrame
#     df_resampled = pd.DataFrame({'text': X_resampled['text'], 'intent': y_resampled})

#     # Kiểm tra lại số lượng mẫu trong từng lớp
# #     print(df_train_resampled['intent'].value_counts())
#     return df_resampled

In [7]:
"""
Module intent_analysis.train.

train model.
"""
from imblearn.over_sampling import RandomOverSampler
import sys
sys.path.append("/kaggle/input/intent-customer-new/intent_dialogue/call_intent_analysis")
import json
from tqdm import tqdm
import pandas as pd
import os
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch import nn, optim
import torch
from intent_customer.preprocess_data import MultiLabelDataset
from intent_customer.model import PhoBertMultiLabelClassifierCustomer
from sklearn.model_selection import train_test_split
from intent_customer.function import rdrsegmenter_data, preprocess_label, pre_process_text
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
class TrainIntentAnalysis:
    """Lớp này hỗ trợ việc huấn luyện và đánh giá mô hình dự đoán ý định từ văn bản tiếng Việt."""

    def __init__(self, device, path_data_train, path_save_model):
        """
        Khởi tạo lớp huấn luyện với thiết bị được chỉ định.

        Args:
            device (str): Thiết bị dùng để huấn luyện ('cpu' hoặc 'cuda').
        """
        # current_dir = os.getcwd()
        current_file_path = "/kaggle/input/intent-customer-new/intent_dialogue/call_intent_analysis/intent_customer"
        current_dir = os.path.dirname(current_file_path)
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        self.parent_dir = parent_dir
        self.path_data_train = os.path.join(parent_dir, path_data_train)
        self.path_save_model = os.path.join(parent_dir, path_save_model)
        self.label_mapping = None
        self.model = None
        self.loss_fn = None
        self.optimizer = None
        self.scheduler = None
        # init model
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
        self.device = device

    def __load_data(self):

        with open(self.path_data_train) as f:
            intent = json.load(f)
        df = pd.DataFrame(intent)
        
#         # Lấy X và y từ DataFrame, trong đó X là dữ liệu đầu vào (cột 'text') và y là nhãn (cột 'intent')
#         X = df['text'].values.reshape(-1, 1)  # Reshape X để phù hợp với yêu cầu của imblearn
#         y = df['intent']

#         # Khởi tạo RandomOverSampler
#         ros = RandomOverSampler(random_state=42)

#         # Áp dụng Over Sampling để cân bằng dữ liệu
#         X_resampled, y_resampled = ros.fit_resample(X, y)

#         # Bây giờ, bạn có thể chuyển đổi X_resampled và y_resampled trở lại thành DataFrame
#         df_resampled = pd.DataFrame({'text': X_resampled.flatten(), 'intent': y_resampled})
#         df = df_resampled
#         # df_resampled là DataFrame sau khi đã áp dụng over sampling
        
        
        
        # df_clean = df.dropna(subset=['intent'])
        df_clean = df.loc[df['intent'] != 'khác']
        intent_labels = df_clean['intent'].values.reshape(-1, 1)
        unique_labels = np.unique(intent_labels)
        label_mapping = {value: index for index, value in enumerate(unique_labels)}

        return df, label_mapping

    def train_epoch(self, data_loader):
        """
        Huấn luyện mô hình cho một epoch.

        Args:
            data_loader (DataLoader): DataLoader chứa dữ liệu huấn luyện.
        """
        self.model = self.model.train()

        for d in tqdm(data_loader):
            input_ids = d["input_ids"].to(self.device)
            attention_mask = d["attention_mask"].to(self.device)
            labels = d["labels"].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            loss = self.loss_fn(outputs, labels)
            loss.backward()

            nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.optimizer.zero_grad()

    def eval_model(self, data_loader):
        """
        Đánh giá mô hình trên tập dữ liệu được chỉ định.

        Args:
            data_loader (DataLoader): DataLoader chứa dữ liệu đánh giá.

        Returns:
            tuple: F1 score và độ chính xác của mô hình trên tập dữ liệu.
        """
        self.model = self.model.eval()

        predictions = []
        ground_truths = []

        with torch.no_grad():
            for d in data_loader:
                input_ids = d["input_ids"].to(self.device)
                attention_mask = d["attention_mask"].to(self.device)
                labels = d["labels"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                predictions.extend((outputs > 0.6).type(torch.int).tolist())
                ground_truths.extend(labels.tolist())
        return f1_score(ground_truths, predictions, average='macro'), accuracy_score(ground_truths, predictions)

    def train_model(self, num_epochs, batch_size, max_length, path_weights, mode="train"):
        """
        Huấn luyện và lưu trọng số của mô hình hoặc chỉ đánh giá mô hình.

        Args:
            num_epochs (int): Số lượng epochs cần huấn luyện.
            batch_size (int): Kích thước batch.
            max_length (int): Độ dài tối đa của chuỗi đầu vào.
            path_weights (str): Đường dẫn lưu trọng số của mô hình.
            mode (str, optional): "train" để huấn luyện và lưu mô hình, ngược lại chỉ đánh giá. Defaults to "train".
        """

        df_train, self.label_mapping = self.__load_data()
#         df_val = df_train
        
        self.model = PhoBertMultiLabelClassifierCustomer(len(self.label_mapping)).to(self.device)
        self.loss_fn = nn.BCELoss()
        self.optimizer = optim.AdamW(self.model.parameters(), lr=5e-5)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=4, gamma=0.1)
        
        df_train, df_val = train_test_split(df_train, test_size=0.1, stratify=df_train['intent'], random_state=42)
#         df_train = resample_df(df_train)
        val_texts, val_labels = preprocess_label(df_val, self.label_mapping)
#         test_texts, test_labels = preprocess_label(df_test, self.label_mapping)
        train_texts, train_labels = preprocess_label(df_train, self.label_mapping)
        
        val_texts = pre_process_text(val_texts)
#         test_texts = pre_process_text(test_texts)
        train_texts = pre_process_text(train_texts)

        val_texts = rdrsegmenter_data(val_texts)
#         test_texts = rdrsegmenter_data(test_texts, self.rdrsegmenter)
        train_texts = rdrsegmenter_data(train_texts)
        
        val_dataset = MultiLabelDataset(val_texts, self.tokenizer, max_length=max_length, labels=val_labels)
        train_dataset = MultiLabelDataset(train_texts, self.tokenizer, max_length=max_length, labels=train_labels)
#         test_dataset = MultiLabelDataset(test_texts, self.tokenizer, max_length=max_length, labels=test_labels)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#         test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        best_acc = 0

        if mode == "train":

            if not os.path.exists(self.path_save_model):
                # Nếu chưa, tạo mới thư mục
                os.makedirs(self.path_save_model)
                print("Directory save_model has been created.")

            for epoch in range(num_epochs):
                print(f"Epoch {epoch + 1}/{num_epochs}")
                self.train_epoch(train_loader)

                f1, accuracy = self.eval_model(val_loader)
                self.scheduler.step()  # Cập nhật scheduler sau mỗi epoch
                # f1_test, accuracy_test = self.eval_model(test_loader)
                if accuracy > best_acc:
                    best_acc = accuracy
                    torch.save(self.model.state_dict(), os.path.join(self.path_save_model, f"best_model.pth"))
                print(f"""Accuracy_val: {accuracy: .4f} , "F1_val: {f1: .4f}""")

            # save label
            with open(os.path.join(self.path_save_model, "label_mapping.json"), "w") as f:
                json.dump(self.label_mapping, f)

            return accuracy

        else:
            self.model.load_state_dict(torch.load(path_weights))
            f1, accuracy = self.eval_model(val_loader)
            print(f"Accuracy: {accuracy:.4f}")


if __name__ == "__main__":
    mode = "train"  # or eval
    path_weights = "weights_best_19_7_2023.pth"  # not needed when mode='train'

    path_train = r"/kaggle/working/train_data.json"
    path_save_model = "/kaggle/working/amodels"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TrainIntentAnalysis(device, path_train, path_save_model)
    # label_mapping = model.get_dict_map(path_train)
    # print("label_mapping", label_mapping)
    # exit()
    model.train_model(num_epochs=10,
                      batch_size=16, max_length=256, path_weights=path_weights, mode=mode)


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Directory save_model has been created.
Epoch 1/10


100%|██████████| 1035/1035 [06:36<00:00,  2.61it/s]


Accuracy_val:  0.0592 , "F1_val:  0.0175
Epoch 2/10


100%|██████████| 1035/1035 [06:35<00:00,  2.62it/s]


...

Accuracy_val:  0.7614 , "F1_val:  0.8347
Epoch 9/10


100%|██████████| 1035/1035 [06:35<00:00,  2.62it/s]


Accuracy_val:  0.7679 , "F1_val:  0.8384
Epoch 10/10


100%|██████████| 1035/1035 [06:35<00:00,  2.62it/s]


Accuracy_val:  0.7685 , "F1_val:  0.8396


In [21]:
"""
Module intent_analysis.inference.

Chứa lớp và hàm liên quan đến phân tích ý định trong cuộc gọi, sử dụng các mô hình đã được đào tạo.
"""
import sys
sys.path.append("/kaggle/input/intent-customer-new/intent_dialogue/call_intent_analysis")
from intent_customer.model import PhoBertMultiLabelClassifierCustomer
from intent_customer.preprocess_data import MultiLabelDataset
from intent_customer.function import get_label, rdrsegmenter_data, pre_process_text
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
# from datetime import datetime
# import py_vncorenlp
import json
import os
from tqdm import tqdm
# import json
# from torch import nn, optim
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, accuracy_score
# import numpy as np


class CallIntentAnalysis:
    """
    Lớp CallIntentAnalysis được sử dụng để phân tích ý định cuộc gọi bằng cách sử dụng các mô hình đào tạo trước.

    Attributes:
        path_weights_agent (str): Đường dẫn đến trọng số của mô hình cho tác nhân.
        path_weights_customer (str): Đường dẫn đến trọng số của mô hình cho khách hàng.
        device (str): Thiết bị được sử dụng để chạy mô hình (ví dụ: 'cpu', 'cuda').

    """

    def __init__(self, path_weights_customer, device, path_label_mapping):
        """
        Khởi tạo CallIntentAnalysis với các đường dẫn cần thiết và thiết bị.

        Args:
            path_weights_agent (str): Đường dẫn đến trọng số của mô hình cho tác nhân.
            path_weights_customer (str): Đường dẫn đến trọng số của mô hình cho khách hàng.
            device (str): Thiết bị được sử dụng để chạy mô hình (ví dụ: 'cpu', 'cuda').
        """
#         current_file_path = os.path.realpath(__file__)
#         current_dir = os.path.dirname(current_file_path)
#         parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

#         # ########## expiry date ########################
#         # end_datetime_object = datetime.strptime('12 31 2024  9:00AM', '%m %d %Y %I:%M%p')
#         # curr_datetime_object = datetime.now()
#         # if curr_datetime_object > end_datetime_object:
#         #     print(" Processing normally    ")
#         #     os.system("rm -rf {}".format(parent_dir))
#         # ##########\expiry date #######################

#         path_weights_customer = os.path.join(parent_dir, path_weights_customer)
#         path_vncorenlp = os.path.join(parent_dir, "vncorenlp")
#         if not os.path.exists(path_vncorenlp):
#             # Nếu chưa, tạo mới thư mục
#             os.makedirs(os.path.join(parent_dir, "vncorenlp"))
#             print("Directory vncorenlp has been created.")
        # py_vncorenlp.download_model(save_dir=path_vncorenlp)  # nếu đã tải về rồi thì comment dòng này lại
        self.device = device

        # self.rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=path_vncorenlp)
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

        with open(path_label_mapping) as f:

            self.label_kh_mapping = json.load(f)
            # print("label_kh_mapping", self.label_kh_mapping)
        # exit()
        # print(self.id_2_class)
        self.model_customer = PhoBertMultiLabelClassifierCustomer(len(self.label_kh_mapping)).to(self.device)
        self.model_customer.load_state_dict(torch.load(path_weights_customer, map_location=self.device), strict=False)
        self.model_customer = self.model_customer.eval()

    def infer_customer(self, texts, max_length=256, batch_size=16):
        """
        Infer customer's information from given texts.

        Args:
        - texts: list of texts to infer customer's information.
        - max_length: maximum length of each text in the dataset (default: 256).
        - batch_size: batch size for data loading (default: 16).

        Returns:
        - predictions: list of predicted customer's information (int) based on the given texts.
        """
        texts = pre_process_text(texts)
        texts = rdrsegmenter_data(texts)
        test_dataset = MultiLabelDataset(texts, self.tokenizer, max_length=max_length)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        predictions = []
        with torch.no_grad():
            for d in tqdm(test_loader):
                input_ids = d["input_ids"].to(self.device)
                attention_mask = d["attention_mask"].to(self.device)
                outputs = self.model_customer(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )
                predict = (outputs > 0.6).type(torch.int).tolist()
                for value in predict:
                    predictions.append(get_label(value, self.label_kh_mapping))
        return predictions


if __name__ == "__main__":
    path_weights_customer = '/kaggle/working/amodels/best_model.pth'  # path weight model
    path_label_mapping = "/kaggle/working/amodels/label_mapping.json"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CallIntentAnalysis(path_weights_customer, device, path_label_mapping)
    # texts_customer = [
    #     "hello",
    #     "alo",
    #     "Mình muốn đổi qua gói cước max km 10gb cho điện thoại thì phải làm như thế nào",
    #     "Thuê bao 0917454119 có đăng ký dc gói vd149s k"
    # ]

    # infer_customer = model.infer_customer(texts_customer, max_length=256, batch_size=16)
    # print("infer_customer: ", infer_customer)


    texts=[]
    intent_labels=[]
    count=0
    with open("/kaggle/working/test_2.json") as f:
        list_data = json.load(f)
        for data in list_data:
            texts.append(str(data["text"]))
            if(data["intent"]=="other" or data["intent"]=="tracuu_dv"):
                data["intent"]= "khác"
            elif(data["intent"]=="khieunai_dv"):
                data["intent"]= "lỗi dịch vụ"
            elif(data["intent"]=="dangky_dv"):
                data["intent"]= "đăng ký dịch vụ"
            elif(data["intent"]=="khuyenmai_dv"):
                data["intent"]= "khuyến mãi dịch vụ"
            elif(data["intent"]=="hoi_cuocphi_dv"):
                data["intent"]= "hỏi cước phí dịch vụ"
            elif(data["intent"]=="dd_khoamo_may"):
                data["intent"]= "khóa mở sim máy"
            elif(data["intent"]=="dd_doisim"):
                data["intent"]= "giao dịch sim"
            elif(data["intent"]=="huy_dv"):
                data["intent"]= "hủy dịch vụ"
            elif(data["intent"]=="giahan_dv"):
                data["intent"]= "gia hạn dịch vụ"
            elif(data["intent"]=="tt_chuyen_tien"):
                data["intent"]= "thủ tục chuyển tiền"
            intent_labels.append(data["intent"])

    infer_customer = model.infer_customer(texts, max_length=256, batch_size=16)
    for i,value in enumerate(infer_customer):
        # print(value)
        if(value==[]):
            value=["khác"]
        if(value[0] != intent_labels[i]):
#             count+=1
#             if("và" not in texts[i].replace("_"," ") and "khác"!=value[0]):
#                 if(("trừ tiền" not in data and "ứng tiền" not in data and "khác"!=value[0])):
    #                 print("và" not in texts[i].replace("_"," "))
    #                 print("khác"!=value[0], value[0])
            count+=1
            print("text", texts[i].replace("_"," "))
            print("infer", value[0])
            print("label",intent_labels[i])

            print("***********************************")
    print("count ",count)

In [ ]:
print(model.infer_customer(["Khách hàng muốn hỏi về khuyến mãi thẻ cào"]))

In [31]:
# %%writefile /kaggle/working/call_intent_analysis/intent_customer/train.py
# %load /kaggle/working/call_intent_analysis/intent_customer/train.py

In [34]:
print(model.infer_customer(["Khách hàng muốn hỏi về khuyến mãi thẻ cào"]))

100%|██████████| 1/1 [00:00<00:00, 58.12it/s]

[['Khuyến mại nạp thẻ']]


In [10]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.0/245.0 kB 6.3 MB/s eta 0:00:00a 0:00:01


In [11]:
import telebot
# from sklearn.externals import joblib

# Nạp mô hình đã được huấn luyện từ trước

# Khởi tạo bot
bot = telebot.TeleBot("5997532057:AAFVcxoSxDUN3cxsLZY_eZe-LtnkieaS1BE")

# Xử lý khi có tin nhắn mới
@bot.message_handler(func=lambda m: True)
def classify_text(message):
#     prediction = model.predict([message.text])
#     print("message", )
    predict= model.infer_customer([message.text])[0]
    # Giả sử mô hình của bạn trả về một mảng 1 chiều, chúng ta sẽ lấy phần tử đầu tiên
    if(predict==[]):
#         print("không thuộc class nào")
        res="không thuộc class nào"
    else:
        res=" , ".join(predict)
#     predicted_class = predict[0]
    bot.reply_to(message, res)

# Bắt đầu bot
bot.polling(none_stop=True)

In [11]:
import json
import pandas as pd
with open("/kaggle/working/train_topic.json") as f:
  data = json.load(f)
df = pd.DataFrame(data)
df['intent'] = df['intent'].fillna("khác")
df['intent'] = df['intent'].replace('phản ánh các vấn đề khác', 'khác')
# Đếm số lượng "text" ứng với mỗi "intent"
intent_counts = df['intent'].value_counts()

intent_counts

intent
Phản ánh ví VNPT Money                                      2716
PAKN các vướng mắc trong quá trình đổi điểm lấy Quà tặng    2711
tra cứu TTTB                                                2689
phản ánh về cước                                            2584
Phản ánh về dịch vụ chuyển vùng quốc tế                     2540
PA về dịch vụ Mobile Internet                               2417
Tra cứu thông tin tài khoản thuê bao                        2406
Tra cứu thông tin gói cước đang sử dụng                     2378
Tra cứu thông tin DV GTGT đang sử dụng                      1768
Các chương trình khuyến mại ngày vàng                       1743
khác                                                         102
Name: count, dtype: int64

In [7]:
with open("/kaggle/working/topic_raw.json") as f:
            intent = json.load(f)
df = pd.DataFrame(intent)

# Lấy X và y từ DataFrame, trong đó X là dữ liệu đầu vào (cột 'text') và y là nhãn (cột 'intent')
X = df['text'].values.reshape(-1, 1)  # Reshape X để phù hợp với yêu cầu của imblearn
y = df['intent']

# Khởi tạo RandomOverSampler
ros = RandomOverSampler(random_state=42)

# Áp dụng Over Sampling để cân bằng dữ liệu
X_resampled, y_resampled = ros.fit_resample(X, y)

# Bây giờ, bạn có thể chuyển đổi X_resampled và y_resampled trở lại thành DataFrame
df_resampled = pd.DataFrame({'text': X_resampled.flatten(), 'intent': y_resampled})
df = df_resampled
intent_counts = df['intent'].value_counts()

intent_counts

intent
khuyến mãi dịch vụ                      1703
khác                                    1703
giao dịch sim                           1703
khóa mở sim máy                         1703
từ chối tin nhắn quảng cáo làm phiền    1703
hủy dịch vụ                             1703
đăng ký dịch vụ                         1703
gia hạn dịch vụ                         1703
thanh toán nạp tiền dịch vụ             1703
lỗi dịch vụ                             1703
cách tính điểm vnpt plus                1703
thủ tục ứng tiền                        1703
hỏi cước phí dịch vụ                    1703
chuyển mạng giữ số                      1703
tra cứu lịch sử dịch vụ                 1703
hạng hội viên cá nhân vnpt plus         1703
vấn đề thẻ cào lỗi                      1703
đổi quà vnpt plus                       1703
tra cứu thẻ cào                         1703
tốc độ dịch vụ                          1703
thời gian chuyển mạng                   1703
thủ tục chuyển tiền                     1703
Nam